## 一元线性回归

线性回归是一种统计方法，用于研究两个或多个变量之间的线性关系。其目的是通过拟合一条直线来预测因变量（目标变量）与自变量（特征变量）之间的关系。线性回归模型可以表示为：

$$ y = wx + b $$

其中：
- $ y $ 是因变量
- $ x $ 是自变量
- $ w $ 是斜率（表示自变量对因变量的影响程度）
- $ b $ 是截距（表示当自变量为零时，因变量的值）

线性回归广泛应用于经济学、金融学、社会科学和自然科学等领域，用于预测和解释变量之间的关系。

### 基础环境配置

In [1]:
import numpy
import matplotlib.pyplot
import pandas

matplotlib.pyplot.style.use("matplotlib.mplstyle")

### 读取 CSV 数据集

In [2]:
data = pandas.read_csv("data/202501061035.csv")

X = (data["size"]/1000).to_numpy()
Y = (data["price"]/1000).to_numpy()

### 输出统计坐标系

In [ ]:
matplotlib.pyplot.scatter(X, Y, marker="x", c="r")
matplotlib.pyplot.title("Example House Prices")
matplotlib.pyplot.ylabel("Price (in 1000s of dollars)")
matplotlib.pyplot.xlabel("Size (1000 sqft)")
matplotlib.pyplot.show()

### 一元线性回归相关公式定义

|名称|定义|
|-|-|
|一阶线性函数|$ f_{w,b}(x^{(i)})=wx^{(i)}+b $|
|均方误差公式|$ MSE(w,b)=\frac{1}{2n}\sum_{i=1}^n(f_{w,b}(x^{(i)}) - \hat{y}^{(i)})^2 $|
|梯度下降公式|$ w=w-\alpha\cdot\frac{\partial{MSE(w,b)}}{\partial{w}},b=b-\alpha\cdot\frac{\partial{MSE(w,b)}}{\partial{b}} $|

此处涉及偏导数的推导：
$$ \frac{\alpha{MSE(w,b)}}{\alpha{w}}=\frac{1}{2n}\sum_{i=1}^n\frac{\alpha}{\alpha{w}}[(wx^{(i)}+b-y^{(i)})^2] $$
根据微积分的链式法则：
$$ y=f(g(x))=>\frac{dy}{dx}=\frac{df}{dg}\cdot\frac{dg}{dx} $$
将下面的推导带入：
$$
g(x^{(i)})=wx^{(i)}+b-y^{(i)}, \\
MSE(w,b)=\frac{1}{2n}\sum_{i=1}^n(g(x^{(i)}))^2 \\
=>w=w-\alpha\cdot\frac{\partial{MSE(w,b)}}{\partial{w}}=w-\alpha\cdot\frac{1}{n}\sum_{i=1}^n(wx^{(i)} + b - \hat{y}^{(i)})\cdot{x^{(i)}}
$$

In [4]:
def compute_mse(X, Y, W, B):
    n = X.shape[0]
    f_wb = W[..., numpy.newaxis] * X + B[..., numpy.newaxis]
    mse = numpy.sum((f_wb - Y) ** 2, axis=-1) / (2 * n)
    return mse

### 梯度值计算

In [5]:
def compute_gradient(X, Y, w, b):
    n = X.shape[0]
    f_wb = w * X + b
    dw = numpy.sum((f_wb - Y) * X) / n
    db = numpy.sum(f_wb - Y) / n
    return dw, db


### 梯度下降计算

In [6]:
import math

def compute_gradient_descent(X, Y, alpha, iters):
    tw, tb = 500, 300
    tws, tbs = [], []
    for i in range(iters):
        dw, db = compute_gradient(X, Y, tw, tb)
        tw = tw - alpha * dw
        tb = tb - alpha * db
        if i % math.ceil(iters / 10) == 0:
            tws.append([tw])
            tbs.append([tb])
            print(f"w: {tw: 0.3e}, b:{tb: 0.5e}")
    tws = numpy.array(tws)
    tbs = numpy.array(tbs)
    mse = compute_mse(X, Y, tws, tbs)
    points = numpy.column_stack((tws, tbs, mse))
    return tw, tb, points


### 执行训练

In [ ]:
fw, fb, points = compute_gradient_descent(X, Y, 0.01, 100)

print(f"(w,b) found by gradient descent: ({fw:8.4f},{fb:8.4f})")

### 绘制计算过程

In [ ]:
%config InlineBackend.print_figure_kwargs = {"bbox_inches": None}

W = numpy.arange(-100, 500, 5)
B = numpy.arange(-300, 300, 5)
W, B = numpy.meshgrid(W, B)
Z = compute_mse(X, Y, W, B)

figure = matplotlib.pyplot.figure()
axes = figure.add_subplot(projection="3d")
surface = axes.plot_surface(W, B, Z, cmap=matplotlib.cm.Blues, alpha=0.75)

axes.view_init(elev=25, azim=145, roll=0)
axes.set_title("Gradient Descent")
axes.set_xlabel("W Values")
axes.set_ylabel("B Values")
axes.set_zlabel("MSE Values", labelpad=10)

tws, tbs, mse = points[:, 0], points[:, 1], points[:, 2]
axes.plot3D(tws, tbs, mse, color="red", linewidth=2, label="Gradient Descent Path")
axes.legend(bbox_to_anchor=(2, 0.5))
axes.set_box_aspect([2, 2, 1]) 

matplotlib.pyplot.show()

In [ ]:
matplotlib.pyplot.scatter(X, Y, marker="x", c="r")
matplotlib.pyplot.plot(X, fw * X + fb, color="blue", label=f"y = {fw}x + {fb}")
matplotlib.pyplot.title("Example House Prices")
matplotlib.pyplot.ylabel("Price (in 1000s of dollars)")
matplotlib.pyplot.xlabel("Size (1000 sqft)")
matplotlib.pyplot.show()